In [56]:
import pandas as pd
import sqlalchemy
import os
import re
import json
import numpy as np
import math
from dotenv import load_dotenv

load_dotenv()

True

In [57]:
# Gunakan ini jika ingin mengambil data dari database

mysql_host = os.getenv("mysql_host")
mysql_username = os.getenv("mysql_username")
mysql_password = os.getenv("mysql_password")
mysql_port = os.getenv("mysql_port")
mysql_db_name_movie = os.getenv("mysql_db_name_movie")
mysql_db_name_rating = os.getenv("mysql_db_name_rating")

engine_movie = sqlalchemy.create_engine(
    f'mysql+mysqlconnector://{mysql_username}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_db_name_movie}',
    echo=False)

engine_rating = sqlalchemy.create_engine(
    f'mysql+mysqlconnector://{mysql_username}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_db_name_rating}',
    echo=False)

In [58]:
sql_movie = """SELECT * FROM movie"""
sql_rating = """SELECT * FROM rating"""

# Hilangkan komentar pada kode di bawah ini untuk mengambil data menggunakan MySQL

# data_movie = pd.read_sql(sql_movie, engine_movie)
# data_rating = pd.read_sql(sql_rating, engine_rating)
# data_rating

In [59]:
# Kali ini menggunakan data dalam format .csv agar lebih cepat diproses
data_movie = pd.read_csv("movie.csv")
data_rating = pd.read_csv("rating.csv")

In [60]:
data_movie

,movieId,title,genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27270,131254,Kein Bund für's Leben (2007),Comedy
27271,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27272,131258,The Pirates (2014),Adventure
27273,131260,Rentun Ruusu (2001),(no genres listed)


In [61]:
data_rating

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [62]:
# Berikut ini adalah kode untuk mendapatkan rata-rata rating per movieId

avg_movie_rating = data_rating.groupby('movieId', as_index=False)['rating'].mean()
avg_movie_rating = avg_movie_rating.rename(columns = {'rating':'rating_avg', 'genres':'genre'}, inplace = False)
avg_movie_rating['rating_avg'] = avg_movie_rating['rating_avg'].round(decimals=1)
avg_movie_rating

,movieId,rating_avg
0,1,3.9
1,2,3.2
2,3,3.2
3,4,2.9
4,5,3.1
...,...,...
26739,131254,4.0
26740,131256,4.0
26741,131258,2.5
26742,131260,3.0


In [63]:
# Mengubah nama kolom dari 'genres' menjadi 'genre'
data_movie = data_movie.rename(columns = {'genres':'genre'}, inplace = False)

In [64]:
# Fungsi untuk mentransformasi genre menjadi format json array

def transform_genre(genre):
    if genre == "" or genre is None:
        return None
    
    if '|' not in genre:
        return json.dumps([genre])
    
    return json.dumps(genre.split("|"))
    
transform_genre("Adventure|Fantasy|Horror")

'["Adventure", "Fantasy", "Horror"]'

In [65]:
# Fungsi untuk mendapatkan tahun film dari judul film / title

def get_year(title):
    if ')' not in title[-6:]:
        return 0
    
    # rm_brackets = re.search(f"\(.*?\)", title[-6:])
    rm_brackets = re.findall(f"\d", title[-6:])
    
    if len(rm_brackets) != 4:
        return None
    
    return int(''.join(rm_brackets))

get_year("Toy Story (1992)")

1992

In [66]:
# Fungsi untuk membersihkan judul film / title

def clean_title(title):
    if ')' not in title and '(' not in title:
        return title.strip()
    
    rm_brackets = re.search(f"\(.*?\)", title)
    title = title.replace(rm_brackets[0], "")
    
    return title.strip()

clean_title("Toy Story (19912312)")

'Toy Story'

In [67]:
# Proses transformasi judul film, genre, dan juga tahun film

movie_joined = pd.merge(data_movie, avg_movie_rating, how="left", on="movieId")
movie_joined = movie_joined.fillna(0)
movie_joined['title'] = movie_joined['title'].apply(lambda x: x.strip())
movie_joined['year'] = movie_joined['title'].apply(lambda x: get_year(x))
movie_joined['title'] = movie_joined['title'].apply(lambda x: clean_title(x))
movie_joined['genre'] = movie_joined['genre'].apply(lambda x: x.strip())
movie_joined['genre'] = movie_joined['genre'].apply(lambda x: x.replace("(no genres listed)", ""))
movie_joined['genre'] = movie_joined['genre'].apply(lambda x: transform_genre(x))
movie_joined['year'] = movie_joined['year'].fillna(0)
movie_joined = movie_joined.astype({'year':'Int64'})
movie_joined.to_csv("movie_rating.csv", index=False)
movie_joined

,movieId,title,genre,rating_avg,year
0,1,Toy Story,"[""Adventure"", ""Animation"", ""Children"", ""Comedy...",3.9,1995
1,2,Jumanji,"[""Adventure"", ""Children"", ""Fantasy""]",3.2,1995
2,3,Grumpier Old Men,"[""Comedy"", ""Romance""]",3.2,1995
3,4,Waiting to Exhale,"[""Comedy"", ""Drama"", ""Romance""]",2.9,1995
4,5,Father of the Bride Part II,"[""Comedy""]",3.1,1995
...,...,...,...,...,...
27270,131254,Kein Bund für's Leben,"[""Comedy""]",4.0,2007
27271,131256,"Feuer, Eis & Dosenbier","[""Comedy""]",4.0,2002
27272,131258,The Pirates,"[""Adventure""]",2.5,2014
27273,131260,Rentun Ruusu,None,3.0,2001


In [68]:
# Funsi untuk melakukan Insert data baru dan memperbaharui data yang sudah ada ke dalam database MySQL (harusnya disimpan ke Data Warehouse)

def upsert_data(movie_id, title, genre, rating_avg, year):
    
    if year == 0:
        year = "NULL"
    
    query_upsert = f"""INSERT INTO `movie_rating` VALUE ({movie_id}, "{title}", '{genre}', 
                    {rating_avg}, {year}) ON DUPLICATE KEY UPDATE movieId={movie_id}, title="{title}", genre='{genre}', 
                    rating_avg={rating_avg}, year={year}"""
    
    try:
        with engine_movie.connect() as conn_movie:
            result = conn_movie.execute(sqlalchemy.text(query_upsert))
        
    except sqlalchemy.exc.SQLAlchemyError as e:
        error = str(e)
        print(error)

for index, row in movie_joined.iterrows():
    upsert_data(row['movieId'], row['title'], row['genre'], row['rating_avg'], row['year'])
    
print("All Tasks is Done!")

All Tasks is Done!
